In [ ]:
import pandas as pd
import numpy as np
import timeit
from sklearn import datasets, linear_model
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from scipy.sparse import csr_matrix, hstack
import lightgbm as lgb
start = timeit.default_timer()
train = pd.read_csv('../input/train.tsv', sep = '\t')
test = pd.read_csv('../input/test_stg2.tsv', sep = '\t')
print ('Train shape :', train.shape)
print ('Test shape :' , test.shape)
nrow_train = train.shape[0]
y = np.log1p(train['price'])
merge = pd.concat([train, test])
merge['category_name'].fillna(value = 'missing', inplace = True)
merge['brand_name'].fillna(value = 'missing', inplace = True)
merge['item_description'].fillna(value = 'missing', inplace = True)
cv = CountVectorizer(min_df= 10)
X_name = cv.fit_transform(merge['name'])
dv = CountVectorizer()
X_category = dv.fit_transform(merge['category_name'])
tv = TfidfVectorizer(max_features= 20, ngram_range = (1,3), stop_words = 'english')
X_description = tv.fit_transform(merge['item_description'])
lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(merge['brand_name'])
X_dummies = csr_matrix(pd.get_dummies(merge[['item_condition_id', 'shipping']],sparse=True).values)
sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()
X = sparse_merge[:nrow_train]
X_test = sparse_merge[nrow_train:]
#regr = linear_model.LinearRegression()
#regr.fit(X, y)
#preds = regr.predict(X_test)
lasso_linear = linear_model.Lasso(alpha=1.0)
lasso_linear.fit(X, y)
preds = lasso_linear.predict(X_test)
submission = pd.DataFrame()
submission = test[['test_id']]
submission['price'] = np.expm1(preds)
submission.to_csv('submission_linear_top20_v1.csv', index = False)
stop = timeit.default_timer()
print ('time run code : %s' %(stop - start))  
print ('Finish')